# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [157]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [158]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [159]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [160]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [161]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [162]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

7424639919150298909795

## Use this to import default tbears wallet and run the transactions


In [ ]:
password = "test1_Account"
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
print(deployer_wallet.get_address())
icon_service.get_balance(deployer_wallet.get_address())

In [ ]:
contracts_file_custom = "shubh_yeouido_contracts_20210112084238.pkl"

In [ ]:
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

# Deploying all contracts


In [163]:

contracts = {}
deploy = ['addressProvider', 'daoFund','delegation','feeProvider','governance', 'lendingPool', 'lendingPoolCore','lendingPoolDataProvider','liquidationManager','ommToken', 'priceOracle','rewards' ,'snapshot','worker_token']
# deploy = ['sicx']
# for directory in {"score"}:
# deploy= ['rewards', 'sample_token','snapshot','worker_token']
directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "sample_token":
       params = {'_name': "BridgeDollars",'_symbol':'USDb' ,'_decimals': 18}
    elif item == "omm_token":
        params = {'_initialSupply':0, '_decimals': 18}
    elif item == "worker_token":
        params = {'_initialSupply':100, '_decimals': 18}
    elif item == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

    

0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'status': 1}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x28e74ac0a454938f741b6489d4e1f59b9f6abc024108ec202d4875d1231726ae', 'blockHeight': 14533642, 'blockHash': '0xbcbda86465a151459610f979faa9c5e8f4a3d6fe401f1a68f9c3875342063653', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx42f5395649215d0d7f2a8a3

In [164]:
pprint(contracts)

{'addressProvider': 'cx77497caa623b73bb492942081eb809f824b9c82a',
 'daoFund': 'cx60c222c041ba0ff15f5b8a234c66c7a41ac913d5',
 'delegation': 'cxb4dce4e434f53ac95e21986a602aab5d098e6ac8',
 'feeProvider': 'cx42f5395649215d0d7f2a8a30a08bfaf3dd9d3b4c',
 'governance': 'cxeae8835f601f74af9e5df34af8ad5fbda2e21b9d',
 'lendingPool': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
 'lendingPoolCore': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'lendingPoolDataProvider': 'cxaf695954f6e68aa64e6a6395691b8cd6ea2d6baa',
 'liquidationManager': 'cx65be82220d61d9e11a99c437d027c59bc9a76df5',
 'ommToken': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6',
 'priceOracle': 'cxa02a6ad39fd5d27e427a64de67af31be357a10a4',
 'rewards': 'cxe71386cf93b4ce75a6494d73c7a90d9b5646f5fc',
 'snapshot': 'cx9d28c8e96aeda1082611f26dfb8a437d6e456949',
 'worker_token': 'cx6cca4c5412536aee63f0ecee8c11c3e8bd20fcc0'}


## Deployin remaining contracts


In [165]:
for item in deploy:
    if item not in contracts.keys():
        params = {}
        if item == "sample_token":
            params = {'_name': "BridgeDollars",'_symbol':'USDb' ,'_decimals': 18}
        elif item == "omm_token":
            params = {'_initialSupply':0, '_decimals': 18}
        elif item == "worker_token":
            params = {'_initialSupply':100, '_decimals': 18}
        elif item == "sicx":
            params = {'_initialSupply':500000000, '_decimals': 18}
        elif item == "oToken":
            params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
        deploy_transaction = DeployTransactionBuilder()\
            .from_(deployer_wallet.get_address())\
            .to(GOVERNANCE_ADDRESS)\
            .nid(NID)\
            .nonce(100)\
            .content_type("application/zip")\
            .content(gen_deploy_data_content(item))\
            .params(params)\
            .build()

        step_limit = icon_service.estimate_step(deploy_transaction) + 100000
        
        signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
        tx_hash = icon_service.send_transaction(signed_transaction)

        res = get_tx_result(tx_hash)
        print(res)
        contracts[item] = res.get('scoreAddress', '')

## deploying otoken for usdb

In [166]:
#deploy oUSDb
params = {"_name":"OmmUSDbInterestToken","_symbol":"oUSDb"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oUSDb'] = res.get('scoreAddress', '')


contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x0d1345f3ee68dda4f210b0f67ece6c08cf56fd0b39c0a106cc7381cbfdb1f91d', 'blockHeight': 14533790, 'blockHash': '0x264d5041333bbecb26264c0d85500282f17dd502c5bcf7211ac7c447b28bef9b', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx5f6defd660257584ee55792703a0563b085decd8', 'stepUsed': 1167391040, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1167391040, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'addressProvider': 'cx77497caa623b73bb492942081eb809f824b9c82a',
 'daoFund': 'cx60c222c041ba0ff15f5b8a234c66c7a41ac913d5',
 'delegation': 'cxb4dce4e434f53ac95e21986a602aab5d098e6ac8',
 'feeProvider': 'cx42f5395649215d0d7f2a8a30a08bfaf3dd9d3b4c',
 'governance': 'cxeae8835f601f74af9e5df34af8ad5fbda2e21b9d',
 'lendingPool': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
 'lendingPoolCore': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'lendingPoolDataProvider': 'cxaf695954f6e68aa64e6a6395691b8cd6ea2d6baa',
 'liquidationManager': 'cx65be82220d61d9e11a99c437d027c59bc9a76df5',
 'ommToken': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6',
 'priceOracle': 'cxa02a6ad39fd5d27e427a64de67af31be357a10a4',
 'rewards': 'cxe71386cf93b4ce75a6494d73c7a90d9b5646f5fc',
 'snapshot': 'cx9d28c8e96aeda1082611f26dfb8a437d6e456949',
 'worker_token': 'cx6cca4c5412536aee63f0ecee8c11c3e8bd20fcc0',
 'oUSDb': 'cx5f6defd660257584ee55792703a0563b085decd8'}

## Deploying otoken for icx

In [167]:
#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xc42d303fd0bdd1d69824974fcfa29c4c0c984de00cbd4b730b29a3f56c2dce4e', 'blockHeight': 14533797, 'blockHash': '0xa1804e8bc0387a3c2eaed407cfc80ba05e4fe41dc305884cfdd211712d96aeba', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx2747c9ebafd1125980239d9992fa31d9932353f2', 'stepUsed': 1167388440, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1167388440, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'addressProvider': 'cx77497caa623b73bb492942081eb809f824b9c82a',
 'daoFund': 'cx60c222c041ba0ff15f5b8a234c66c7a41ac913d5',
 'delegation': 'cxb4dce4e434f53ac95e21986a602aab5d098e6ac8',
 'feeProvider': 'cx42f5395649215d0d7f2a8a30a08bfaf3dd9d3b4c',
 'governance': 'cxeae8835f601f74af9e5df34af8ad5fbda2e21b9d',
 'lendingPool': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
 'lendingPoolCore': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'lendingPoolDataProvider': 'cxaf695954f6e68aa64e6a6395691b8cd6ea2d6baa',
 'liquidationManager': 'cx65be82220d61d9e11a99c437d027c59bc9a76df5',
 'ommToken': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6',
 'priceOracle': 'cxa02a6ad39fd5d27e427a64de67af31be357a10a4',
 'rewards': 'cxe71386cf93b4ce75a6494d73c7a90d9b5646f5fc',
 'snapshot': 'cx9d28c8e96aeda1082611f26dfb8a437d6e456949',
 'worker_token': 'cx6cca4c5412536aee63f0ecee8c11c3e8bd20fcc0',
 'oUSDb': 'cx5f6defd660257584ee55792703a0563b085decd8',
 'oICX': 'cx2747c9ebafd1125980239d9992fa31d9932353f2'}

## Deploying otoken for iusdc


In [168]:
#deploy oIUSDC
params = {"_name":"IconUSDInterest","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oIUSDC'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xabb27cf53395e02b1ae846a3cbd3b4e4d21510370d9fc3ddd5af8b3df0b18278', 'blockHeight': 14533803, 'blockHash': '0xa5467e435c41bcb9cba8e081dfddb11e345e6d3482cdf091c38c1f10453bd73f', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx0bdd7fd3dd02283a16dc4f1a48cbb9dd8cf8d994', 'stepUsed': 1167392560, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1167392560, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'addressProvider': 'cx77497caa623b73bb492942081eb809f824b9c82a',
 'daoFund': 'cx60c222c041ba0ff15f5b8a234c66c7a41ac913d5',
 'delegation': 'cxb4dce4e434f53ac95e21986a602aab5d098e6ac8',
 'feeProvider': 'cx42f5395649215d0d7f2a8a30a08bfaf3dd9d3b4c',
 'governance': 'cxeae8835f601f74af9e5df34af8ad5fbda2e21b9d',
 'lendingPool': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
 'lendingPoolCore': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'lendingPoolDataProvider': 'cxaf695954f6e68aa64e6a6395691b8cd6ea2d6baa',
 'liquidationManager': 'cx65be82220d61d9e11a99c437d027c59bc9a76df5',
 'ommToken': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6',
 'priceOracle': 'cxa02a6ad39fd5d27e427a64de67af31be357a10a4',
 'rewards': 'cxe71386cf93b4ce75a6494d73c7a90d9b5646f5fc',
 'snapshot': 'cx9d28c8e96aeda1082611f26dfb8a437d6e456949',
 'worker_token': 'cx6cca4c5412536aee63f0ecee8c11c3e8bd20fcc0',
 'oUSDb': 'cx5f6defd660257584ee55792703a0563b085decd8',
 'oICX': 'cx2747c9ebafd1125980239d9992fa31d9932353f2',
 'oIUSDC'

## Other Contract Addresses

In [169]:
# contracts['sicx']="cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281"
# contracts['staking']="cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c"
contracts['sicx']="cxd59e42181ebcc5ab6c9e0803246d1e5170a7aff4"
contracts['staking']="cx634ac19ceaa647bbb06096fbc1e1337adb09361f"
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
contracts['usdb']="cxaa068556df80f9917ef146e889f0b2c4b13ab634"


In [198]:
pprint(contracts)

{'addressProvider': 'cx77497caa623b73bb492942081eb809f824b9c82a',
 'daoFund': 'cx60c222c041ba0ff15f5b8a234c66c7a41ac913d5',
 'delegation': 'cxb4dce4e434f53ac95e21986a602aab5d098e6ac8',
 'feeProvider': 'cx42f5395649215d0d7f2a8a30a08bfaf3dd9d3b4c',
 'governance': 'cxeae8835f601f74af9e5df34af8ad5fbda2e21b9d',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
 'lendingPoolCore': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'lendingPoolDataProvider': 'cxaf695954f6e68aa64e6a6395691b8cd6ea2d6baa',
 'liquidationManager': 'cx65be82220d61d9e11a99c437d027c59bc9a76df5',
 'oICX': 'cx2747c9ebafd1125980239d9992fa31d9932353f2',
 'oIUSDC': 'cx0bdd7fd3dd02283a16dc4f1a48cbb9dd8cf8d994',
 'oUSDb': 'cx5f6defd660257584ee55792703a0563b085decd8',
 'ommToken': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6',
 'priceOracle': 'cxa02a6ad39fd5d27e427a64de67af31be357a10a4',
 'rewards': 'cxe71386cf93b4ce75a6494d73c7a90d9b5646f5fc',
 'sicx': 'cxd59e42

## Saving the pkl file


In [171]:
now = datetime.utcnow()
filename ="shubh_"+network+f'_contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# filename = "kailash_custom.pkl"
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:

    pkl.dump(contracts, f)

saving contracts list as shubh_yeouido_contracts_20210323082333.pkl


## Updating contracts

In [208]:
contract_ = 'lendingPoolCore'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
elif contract_ == 'oUSDb':
    update = contracts['oUSDb']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update lendingPoolCore cx8bc77f714db402550c02e0fe22093c19d04886fc
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 4 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 8 seconds...
Raised JSONRPCException while returning the custom

{'txHash': '0xd69cb218a0117af4f9e53ede917bde6bf48d87a52f4d921e39a25cbc87413f8f',
 'blockHeight': 14621817,
 'blockHash': '0x03ab08e00aa944c5ef89ed4fc62edceb041aaeed79d7d6bb725ab5b6429b411a',
 'txIndex': 3,
 'to': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'scoreAddress': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'stepUsed': 1876749600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1877100020,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Configuring SCOREs's parameters

In [172]:
settings_lendingPool = [{'contract': 'lendingPool', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                        {'contract': 'lendingPool', 'method': 'setLendingPoolDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
                        {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
                        {'contract': 'lendingPool', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                        {'contract': 'lendingPool', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
                        {'contract': 'lendingPool', 'method': 'setOICX', 'params':{'_address': contracts['oICX']}},
                        {'contract': 'lendingPool', 'method': 'setReward','params': {'_address':contracts['rewards']}},
                        {'contract': 'lendingPool', 'method': 'setSICX','params': {'_address':contracts['sicx']}},
                        {'contract': 'lendingPool', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}}
                       ]
for sett in settings_lendingPool:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setLendingPoolCore, with parameters {'_address': 'cx8bc77f714db402550c02e0fe22093c19d04886fc'} on the lendingPool contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x68fe8de3db3fc39c2fe5ca61428c64403de62bef672683959dec9a32c0cf0a38', 'blockHeight': 14533844, 'blockHash': '0x54046bcb969018664ffea04ac176a30b6072e53899f3530907622eaba04c0d67', 'txIndex': 1, 'to': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905', 'stepUsed': 151320, 'stepPrice': 10000000000, 'cumulativeStepUsed': 151320, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [173]:
settings_lendinPoolDataProvider=[{'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['usdb'],'_sym':"USDb"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setPriceOracle', 'params':{'_address': contracts['priceOracle']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPool', 'params':{'_address': contracts['lendingPool']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationManager', 'params':{'_address': contracts['liquidationManager']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setStaking', 'params':{'_address': contracts['staking']}}
            ]
for sett in settings_lendinPoolDataProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setSymbol, with parameters {'_reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', '_sym': 'USDb'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setSymbol, with parameters {'_reserve': 'cxd59e42181ebcc5ab6c9e0803246d1e5170a7aff4', '_sym': 'ICX'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while 

In [174]:
settings_feeprovider =[{'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}}]
for sett in settings_feeprovider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLoanOriginationFeePercentage, with parameters {'_percentage': 1000000000000000} on the feeProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [175]:
settings_oToken =[{'contract': 'oUSDb', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oUSDb', 'method': 'setReserve', 'params':{'_address':contracts['usdb']}},
            {'contract': 'oUSDb', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oUSDb', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'oUSDb', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}]
for sett in settings_oToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx8bc77f714db402550c02e0fe22093c19d04886fc'} on the oUSDb contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'} on the oUSDb contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -3

In [176]:
setting_priceOracle =[{'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':10*10**17}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'IUSDC','_quote':'USDC','_rate':10*10**17}}

                      ]
for sett in setting_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling set_reference_data, with parameters {'_base': 'USDb', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling set_reference_data, with parameters {'_base': 'ICX', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom respo

In [177]:
setting_addressProvider =[  {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['usdb']}},
                            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oUSDb']}},
                            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
                            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
                            {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
                            {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}},
                            {'contract': 'addressProvider', 'method': 'setOmmToken', 'params':{'_address':contracts['ommToken']}},
                            {'contract': 'addressProvider', 'method': 'setDelegation', 'params':{'_address':contracts['delegation']}},
                            {'contract': 'addressProvider', 'method': 'setRewards', 'params':{'_address':contracts['rewards']}}
                         ]
for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905'} on the addressProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cxaf695954f6e68aa64e6a6395691b8cd6ea2d6baa'} on the addressProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom resp

In [178]:
settings_reserves = [{'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['usdb'],
                                                   "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}},
                     {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sicx'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                    "baseBorrowRate":f"0{'0'*17}",
                                                    "slopeRate1":f"8{'0'*16}",
                                                    "slopeRate2":f"2{'0'*18}"} ]}},
                    {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['iusdc'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}}
                    ]
for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '20000000000000000', 'slopeRate1': '60000000000000000', 'slopeRate2': '1000000000000000000'}]} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxd59e42181ebcc5ab6c9e0803246d1e5170a7aff4', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '000000000000000000', 'slopeRate1': '80000000000000000

In [179]:
settings_oicx = [{'contract': 'oICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                {'contract': 'oICX', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}]
for sett in settings_oicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx8bc77f714db402550c02e0fe22093c19d04886fc'} on the oICX contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxd59e42181ebcc5ab6c9e0803246d1e5170a7aff4'} on the oICX contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cxaf695954f6e68aa64

In [180]:
settings_oiusdc = [ {'contract': 'oIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                    {'contract': 'oIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                    {'contract': 'oIUSDC', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}
                    ]
for sett in settings_oiusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx8bc77f714db402550c02e0fe22093c19d04886fc'} on the oIUSDC contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cx65f639254090820361da483df233f6d0e69af9b7'} on the oIUSDC contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cxaf695954f6e68

In [181]:
settings_liquidationManager =  [{'contract': 'liquidationManager', 'method': 'setLendingPoolDataProvider', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                                {'contract': 'liquidationManager', 'method': 'setLendingPoolCore', 'params': {'_address':contracts['lendingPoolCore']}},
                                {'contract': 'liquidationManager', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                                {'contract': 'liquidationManager', 'method': 'setFeeProvider','params': {'_address':contracts['feeProvider']}},
                                {'contract': 'liquidationManager', 'method': 'setStaking','params': {'_address':contracts['staking']}}
                                ]
for sett in settings_liquidationManager:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolDataProvider, with parameters {'_address': 'cxaf695954f6e68aa64e6a6395691b8cd6ea2d6baa'} on the liquidationManager contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolCore, with parameters {'_address': 'cx8bc77f714db402550c02e0fe22093c19d04886fc'} on the liquidationManager contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setPriceOracle, with parameters {

In [182]:
settings_delegation =[{'contract': 'delegation', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
                     {'contract':  'delegation', 'method': 'setOmmToken','params': {'_address':contracts['ommToken']}}]
for sett in settings_delegation:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx8bc77f714db402550c02e0fe22093c19d04886fc'} on the delegation contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setOmmToken, with parameters {'_address': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6'} on the delegation contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: 

In [183]:
settings_ommToken =[{'contract': 'ommToken', 'method': 'setAdmin','params': {'_admin':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                    {'contract':  'ommToken', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setMinimumStake','params': {'_min':10 * 10**18}},
                    {'contract':  'ommToken', 'method': 'setUnstakingPeriod','params': {'_time': 120}}]
for sett in settings_ommToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setAdmin, with parameters {'_admin': 'cxe71386cf93b4ce75a6494d73c7a90d9b5646f5fc'} on the ommToken contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x0eeb6e5fed471fc2bd032a12d81d644e3338244ae42a326708f03e72a5571261', 'blockHeight': 14534088, 'blockHash': '0xb048b345f56c8ffd5120565d67fbb7625a1dcf3d66bfa0234b18f64ab9e4941e', 'txIndex': 1, 'to': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6', 'stepUsed': 148920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [184]:
settings_lendingPoolCore =[ {'contract': 'lendingPoolCore', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
                            {'contract': 'lendingPoolCore', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                            {'contract': 'lendingPoolCore', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                            {'contract': 'lendingPoolCore', 'method': 'setStaking','params': {'_address':contracts['staking']}},
                            {'contract': 'lendingPoolCore', 'method': 'set_id','params': {'_value':'1'}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['usdb'],'_sym':"USDb"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}}]
for sett in settings_lendingPoolCore:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setLendingPool, with parameters {'_address': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905'} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xd16fc582cc74be8beafc44930d8e2e6c1490b4d99f4d80a548582f0ee2399fb9', 'blockHeight': 14534113, 'blockHash': '0x682575f38f43572ba782a390eaa974b7507d6987ef3f42a0a6aaf2c298c54651', 'txIndex': 2, 'to': 'cx8bc77f714db402550c02e0fe22093c19d04886fc', 'stepUsed': 150520, 'stepPrice': 10000000000, 'cumulativeStepUsed': 674560, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [185]:
settings_priceOracle =[{'contract':  'priceOracle', 'method': 'setBandOracle','params': {'_address':"cx61a36e5d10412e03c907a507d1e8c6c3856d9964"}},
                       {'contract':  'priceOracle', 'method': 'toggleOraclePriceBool','params':{}}]
for sett in settings_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setBandOracle, with parameters {'_address': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling toggleOraclePriceBool, with parameters {} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'

In [39]:
pprint(contracts)

{'addressProvider': 'cx19aa6bc8824297f191d695c00101ecafbce33e16',
 'daoFund': 'cxe328e630d98efeea857ae09d17b61abe9df2ab4d',
 'delegation': 'cxf8ddf4f5972101c3f6eeb944ce8d24ff9828ee23',
 'feeProvider': 'cx1d717762b4abc7859e195d6fddaae4bf671cfd32',
 'governance': 'cx28ce8e8c5905be74657494712df72194cde18a10',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0d6da02f6edd86c4356fb095dd325178da837705',
 'lendingPoolCore': 'cxfbe99218475ad6454206135f9501c76f06b57a55',
 'lendingPoolDataProvider': 'cx09f135dafb11328117b2b03b9bc67bda28b8cab6',
 'liquidationManager': 'cx27b89ee1a3c0606a5bb3a956ebbcafd506b04bfe',
 'oICX': 'cx78fcde79b84b3a727babbb40aa874eabe06ff6e6',
 'oIUSDC': 'cx44c12a1b4aeb62a3e14f73c637241351085477bd',
 'oUSDb': 'cx63514d0ca9c6bbbe4a1df932440cc6d3b68ae8f1',
 'ommToken': 'cxeb7a6f6f4b055dae779ce2c78402cc4b15543057',
 'priceOracle': 'cx0c204884486b02158208b2821cc50362910dfbe7',
 'rewards': 'cxf8ed8f0181e84d9863575f16fa3f8c3921861bef',
 'sicx': 'cxd59e42

In [186]:
settings_snapshot =[{'contract': 'snapshot', 'method': 'setAdmin','params': {'_address':contracts['lendingPool']}},
                    {'contract': 'snapshot', 'method': 'setGovernance','params': {'_address':contracts['governance']}}]
for sett in settings_snapshot:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setAdmin, with parameters {'_address': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905'} on the snapshot contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setGovernance, with parameters {'_address': 'cxeae8835f601f74af9e5df34af8ad5fbda2e21b9d'} on the snapshot contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32

In [187]:
settings_rewards=[ {'contract': 'rewards', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setOmm','params': {'_address':contracts['ommToken']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}},
            {'contract': 'rewards', 'method': 'setAdmin','params': {'_address':contracts['governance']}},
            {'contract': 'rewards', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
            {'contract': 'rewards', 'method': 'setLpToken','params': {'_address':"cx291dacbb875a94b364194a5febaac4e6318681f7"}},
                  
            ]
for sett in settings_rewards:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905'} on the rewards contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setOmm, with parameters {'_address': 'cx9f03e46d637fb0b1c7b539873c7f25db2ddc94e6'} on the rewards contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602

In [188]:
settings_governance =[{'contract': 'governance', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
                      {'contract': 'governance', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                      {'contract': 'governance', 'method': 'setStartTimestamp','params': {}},
                      ]
for sett in settings_governance:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSnapshot, with parameters {'_address': 'cx9d28c8e96aeda1082611f26dfb8a437d6e456949'} on the governance contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x90698f07654801d5f21b36126f001daf80e6a6fba8c9179f53ae47c5ce3a5ee9', 'blockHeight': 14534299, 'blockHash': '0xc06dbcdc6b0b5029b5e108af4ff69cdaffc25978aabd0b5a436572151a689232', 'txIndex': 1, 'to': 'cxeae8835f601f74af9e5df34af8ad5fbda2e21b9d', 'stepUsed': 149920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 149920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding USDb reserve in LendingPoolCore



In [189]:
params ={"_reserve": {"reserveAddress":contracts['usdb'],"oTokenAddress":contracts['oUSDb'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x931f7b6f1b2d75ff8c4f25688097f35118c647139537d2e33e4b97c0518571a3',
 'blockHeight': 14534312,
 'blockHash': '0x96412189f8cb780d9c69ea86ce227146628f0ee7790ec77dc0b146206579ec10',
 'txIndex': 1,
 'to': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'stepUsed': 277040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 277040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding ICX reserve

In [190]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x499b4e89ad8616a29581ef086d0344e1ed509fae187d4f24bf98a133ee2250a6',
 'blockHeight': 14534318,
 'blockHash': '0x593392afe6207a441b28c60e8399cfa0f00db927c98be02d96599db62f9705c1',
 'txIndex': 1,
 'to': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'stepUsed': 276800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding IUSDC reserve

In [191]:
params ={"_reserve": {"reserveAddress":contracts['iusdc'],"oTokenAddress":contracts['oIUSDC'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"6","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x6d9acf96ad1c5dfd9367b299622ce7ab13eb181d927b3bd1349281d518d5e869',
 'blockHeight': 14534324,
 'blockHash': '0x6b9e6d9def4c5c38debc338302b2369ccebcf0488fc61b10ada4528be18a1d2d',
 'txIndex': 1,
 'to': 'cx8bc77f714db402550c02e0fe22093c19d04886fc',
 'stepUsed': 276600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding to Super address provider

In [68]:
sap="cx151289f65d2ccb8191a9efb8fcfc02fa06ce5855"
name="LiquidationTest"

params = { "_env":{
                    "addressProvider":contracts['addressProvider'],
                    "envName":name,
                    "pklName":filename
                    }  
          }

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(sap) \
    .nid(NID) \
    .nonce(100) \
    .method("addEnv")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)


tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x8da6eaaef4295fd5a66b828d6b451655c5e158b4aecd981f66d04290f8d69943',
 'blockHeight': 13624889,
 'blockHash': '0xc2aa95a885481718b64b19b5896500ecebcd60864c82339ee6b96118f963f049',
 'txIndex': 1,
 'to': 'cx151289f65d2ccb8191a9efb8fcfc02fa06ce5855',
 'stepUsed': 191200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 191200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Testing deposit

In [192]:
params = {"_amount": 5 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(5 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x7b259807eac40e300dece858815747d77ab704f66718e52170630aadb524bff3',
 'blockHeight': 14534333,
 'blockHash': '0xab11edee89e86249d6adefb3742b04272ec76826e20ff9fc506274db3cf6c05b',
 'txIndex': 1,
 'to': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
 'stepUsed': 3200260,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 3200260,
 'eventLogs': [{'scoreAddress': 'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx20efebed97d199b8a193a3b7f0769bd3b4d4e905',
    'cx634ac19ceaa647bbb06096fbc1e1337adb09361f',
    '0x4563918244f40000'],
   'data': []},
  {'scoreAddress': 'cx0000000000000000000000000000000000000000',
   'indexed': ['IScoreClaimedV2(Address,int,int)',
    'cx634ac19ceaa647bbb06096fbc1e1337adb09361f'],
   'data': ['0x2e7fbaea8c76bf9018', '0xbe75adbc9d8c47f']},
  {'scoreAddress': 'cxd59e42181ebcc5ab6c9e0803246d1e5170a7aff4',
   'indexed': ['Mint(Address,int,bytes)',
    'cx8bc77f714db402550c02e0fe22093c19d04886fc'],
   '